## 규제선형모델이란? (Regulation linear model)

최적 모델을 위한 cost함수 구성요소는 두가지다
1. 학습데이터 잔차 오류 최소화(min(RSS))
2. 회귀계수 크기 제어($\alpha * ||W||^2_2$)


Cost function goal = $min(RSS(W)) + \alpha * ||W||^2_2$ <br>
그래서 위에 공식에서 알파값이 0과 같이 작아지면 RSS를 최소화 하는 효과가 있고, 알파값이 무한대로 커지면 W를 0에 가깝게 최소화하게 된다. 알파값을 조정해 나가는 과정을 Regulation이라고 한다. 

Regulation에는 L1과 L2가 있다.<br>
L1 regulation : $\alpha * ||W||_1$과 같이 W의 절대값에 대해 페널티를 부여. 영향력이 크지 않은 회귀 계수 값을 0으로 변환<br>
L2 regulation : $\alpha * ||W||^2_2$과 같이 W의 제곱에 대해 페널티를 부여.

### Ridge <br>
Ridge 같은 경우에는 L2를 사용한다.

In [1]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np

/Users/pistispark/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/pistispark/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
# boston 데이터 세트
boston = load_boston()

# to dataframe
bostondf = pd.DataFrame(boston.data, columns = boston.feature_names)

# target = price
bostondf['PRICE'] = boston.target
print(bostondf.shape)
bostondf.info()

(506, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
CRIM       506 non-null float64
ZN         506 non-null float64
INDUS      506 non-null float64
CHAS       506 non-null float64
NOX        506 non-null float64
RM         506 non-null float64
AGE        506 non-null float64
DIS        506 non-null float64
RAD        506 non-null float64
TAX        506 non-null float64
PTRATIO    506 non-null float64
B          506 non-null float64
LSTAT      506 non-null float64
PRICE      506 non-null float64
dtypes: float64(14)
memory usage: 55.5 KB


In [5]:
y_target = bostondf['PRICE']
X_data = bostondf.drop(['PRICE'], axis=1, inplace=False)

# alpha=10으로 설정해 릿지 회귀 수행
ridge = Ridge(alpha=10)
neg_mse_scores = cross_val_score(ridge, X_data, y_target, scoring="neg_mean_squared_error", cv=5)
rmse_scores = np.sqrt(-1 * neg_mse_scores)
avg_rmse = np.mean(rmse_scores)

print(np.round(neg_mse_scores, 3), np.round(rmse_scores, 3), avg_rmse)

[-11.422 -24.294 -28.144 -74.599 -28.517] [3.38  4.929 5.305 8.637 5.34 ] 5.518166280868971


In [9]:
# 릿지에 사용될 alpha 파라미터의 값을 정의
alphas = [0, 0.1, 1, 10, 100]

# alphas list 값을 반복하면서 alpha에 따른 평균 rmse를 구함
for alpha in alphas:
    ridge = Ridge(alpha=alpha)
    
    #cross_val_scores 이용 평균 RMSE 계산
    neg_mse_scores = cross_val_score(ridge, X_data, y_target, scoring="neg_mean_squared_error", cv=5)
    avg_rmse = np.mean(np.sqrt(-1 * neg_mse_scores))
    print('alpha{0} avg_rmse:{1:.3f}'.format(alpha, avg_rmse))

alpha0 avg_rmse:5.829
alpha0.1 avg_rmse:5.788
alpha1 avg_rmse:5.653
alpha10 avg_rmse:5.518
alpha100 avg_rmse:5.330
